In [8]:
from NN.Net import Net
import torch.nn as nn
import torch
from torch.utils.data import TensorDataset, DataLoader

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from common.Database import Database

In [9]:
db = Database()
query = """
    select * 
    FROM habsos_j
    WHERE LATITUDE IS NOT NULL and LONGITUDE IS NOT NULL
    AND SAMPLE_DATE  IS NOT NULL
    and CATEGORY  is not NULL
    and SALINITY  is not NULL
    and WATER_TEMP is not null
    and WIND_DIR is not null 
    and WIND_SPEED is not null;
"""
records, columns = db.execute_query(query)
db.close()
df = pd.DataFrame(records, columns=columns)
df['date'] = pd.to_datetime(df['SAMPLE_DATETIME'])
df['month'] = df['date'].dt.month
le = LabelEncoder()
df['category_encoded'] = le.fit_transform(df['CATEGORY'])

In [10]:
# Select features and target
features = ['LATITUDE', 'LONGITUDE', 'SALINITY', 'WATER_TEMP', 'WIND_DIR', 'WIND_SPEED']
X = df[features]
y = df['category_encoded']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Identify features to scale
features_to_scale = ['SALINITY', 'WATER_TEMP', 'WIND_DIR', 'WIND_SPEED']
scaler = StandardScaler()

# Scale the features
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()
X_train_scaled[features_to_scale] = scaler.fit_transform(X_train[features_to_scale])
X_test_scaled[features_to_scale] = scaler.transform(X_test[features_to_scale])

# Convert the scaled data to NumPy arrays (if not already in that format)
X_train_np = X_train_scaled.values
X_test_np = X_test_scaled.values
y_train_np = y_train.values
y_test_np = y_test.values

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train_np, dtype=torch.float)
X_test_tensor = torch.tensor(X_test_np, dtype=torch.float)
y_train_tensor = torch.tensor(y_train_np, dtype=torch.long)
y_test_tensor = torch.tensor(y_test_np, dtype=torch.long)


train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 64  
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)


In [11]:
input_size = 6
num_classes = 5  
model = Net(input_size, num_classes)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [13]:
num_epochs=128
for epoch in range(num_epochs):
    for X_batch, y_batch in train_loader:
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')



Epoch [10/128], Loss: 0.8787
Epoch [20/128], Loss: 0.5941
Epoch [30/128], Loss: 0.6244
Epoch [40/128], Loss: 0.6263
Epoch [50/128], Loss: 0.5396
Epoch [60/128], Loss: 0.3684
Epoch [70/128], Loss: 0.6279
Epoch [80/128], Loss: 0.5924
Epoch [90/128], Loss: 0.5147
Epoch [100/128], Loss: 0.6257
Epoch [110/128], Loss: 0.6619
Epoch [120/128], Loss: 0.6410


In [15]:
# Collect all predictions and actual labels
all_predictions = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch)
        _, predicted = torch.max(outputs, 1)
        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(y_batch.cpu().numpy())

# Calculate classification metrics
print(classification_report(all_labels, all_predictions))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00        11
           1       0.22      0.16      0.19        50
           2       0.00      0.00      0.00        60
           3       0.87      1.00      0.93      1274
           4       0.00      0.00      0.00       107

    accuracy                           0.85      1502
   macro avg       0.22      0.23      0.22      1502
weighted avg       0.74      0.85      0.79      1502



/Users/tzuhan/opt/miniconda3/envs/capstone/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/tzuhan/opt/miniconda3/envs/capstone/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/tzuhan/opt/miniconda3/envs/capstone/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier